In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from pydrake.all import (
    ControllabilityMatrix,
    DiagramBuilder,
    Linearize,
    LinearQuadraticRegulator,
    MeshcatVisualizer,
    Saturation,
    SceneGraph,
    Simulator,
    StartMeshcat,
    WrapToSystem,
    LinearSystem
)
from pydrake.examples import AcrobotGeometry, AcrobotInput, AcrobotPlant, AcrobotState

meshcat = StartMeshcat()

In [ ]:
def UprightState():
    state = AcrobotState()
    state.set_theta1(np.pi)
    state.set_theta2(0.0)
    state.set_theta1dot(0.0)
    state.set_theta2dot(0.0)
    return state

## Exact Dynamics

In [ ]:
from IPython.display import SVG, display, clear_output, Markdown
from control.models.acrobot import Acrobot2DModel
from control.iLQRController import iLQR

Q = np.diag(np.concatenate(([1.0] * 2, [0.1] * 2)))
Qf = np.diag(np.concatenate(([100.0] * 2, [100.0] * 2)))
R = np.diag([0.01])
dt = 0.05
N = 40

iLQRParams = {
    "Q" : Q,
    "R" : R,
    "Qf" : Qf,
    "N" : N,
    "dt" : dt,
    "max_iter" : 20,
    "max_linesearch_iters": 10,
    "d_tol": 1e-3,
    "interation": "RK4",
}
plant = Acrobot2DModel(**iLQRParams)
controller = iLQR(plant, **iLQRParams)
goal_state  = np.array([np.pi, 0., 0., 0.])
current_state = np.array([0., 0., 0., 0.])
# current_state = np.array([ 2.83752567,  0.7953031,   0.0177701,  -0.2012475 ])
nx, nu, ndx = plant.get_dims()
uref = np.zeros(nu, dtype = np.float64)
xtraj = np.zeros((N, nx))
utraj = np.zeros((N-1, nu))
controller.model.set_references(goal_state, uref)

# print(controller.model.discrete_dynamics(goal_state, 0))
# print(controller.model.xdot_fun(goal_state, 0))
state_error = 0
time = 0

state_log = current_state
action_log = None
while np.linalg.norm(state_error) > 1e-4 or time == 0:
    if not utraj.any():
        utraj[:] = np.random.randn(N-1, nu)*1.0
    else:
        # Use previous utraj as initial guess with shifting 
        # forward a single timestep, 
        utraj[:-1, :] = utraj[1:,:] #+  np.random.randn(N-2, nu)

    xtraj, utraj = controller.run_ilqr(current_state, utraj)
    current_state = xtraj[1, :]
    u = utraj[0, :]
    current_state = np.squeeze(current_state)

    state_error = current_state - goal_state
    state_error[:2] /= np.pi
    terminal_state_error = xtraj[-1, :] - goal_state
    terminal_state_error[:2] /= np.pi
    # print(xtraj)
    clear_output(wait=True)
    display(Markdown(f"**t = {time:4.4f}s:**\n```\nError: {state_error}\n```\n**Control:**\n```\n{u}\n```\n**Terminal State Err:**\n```\n{terminal_state_error}"))
    time += dt

    state_log = np.vstack((state_log, current_state))
    action_log = np.vstack((action_log, u)) if action_log is not None else u


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML  # Import HTML for displaying the animation in Jupyter

def FK(state):
    l1, l2 = 1.0, 2.0
    th1, th2 = state[:2]
    x1 = l1 * np.sin(th1)
    y1 = -l1 * np.cos(th1)
    x2 = l2 * np.sin(th1 + th2) + x1
    y2 = -l2 * np.cos(th1 + th2) + y1
    return x1, y1, x2, y2

n_frames = state_log.shape[0]

# Create the figure and axis for plotting
fig, ax = plt.subplots()
fig.patch.set_facecolor("white")
ax.set_facecolor("white")
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_xlabel("x [m]")
ax.set_ylabel("y [m]")
ax.set_title("iLQR w/ Exact Dynamics")
ax.set_aspect('equal')

# Initialize the plot lines
line1, = ax.plot([], [], 'r-', lw=2)  # First segment (link 1)
line2, = ax.plot([], [], 'b-', lw=2)  # Second segment (link 2)

# Function to initialize the plot (empty frame)
def init():
    line1.set_data([], [])
    line2.set_data([], [])
    return line1, line2

# Function to update the plot for each frame
def update(frame):
    state = state_log[frame, :]
    x1, y1, x2, y2 = FK(state)

    # Update the line data for both segments
    line1.set_data([0, x1], [0, y1])  # First link (from origin to first joint)
    line2.set_data([x1, x2], [y1, y2])  # Second link (from first joint to second joint)
    
    return line1, line2

# Create the animation
ani = FuncAnimation(fig, update, frames=range(n_frames), init_func=init, blit=True, interval=50)
ani.save("/home/malkstik/mnt/results/acrobot_exact_dynamics.gif")
HTML(ani.to_jshtml())
plt.show()

# for state in state_log:
#     x1, y1, x2, y2 = FK(state)
#     print(state[0], state[1])
#     print(x1, y1, x2, y2)
#     print()

## Drake w/ FixValue

In [ ]:
class AcrobotiLQRController2():
    def __init__(self, Q,
                        R,
                        Qf,
                        dt,
                        N,         
                        max_iter,
                        max_linesearch_iters,
                        integration,
                        d_tol,
                        **kwargs
                ):
        model = Acrobot2DModel( Q,
                                Qf,
                                R,
                                dt,
                                N,
                                integration)

        self.controller = iLQR(model,
                                N,         
                                max_iter,
                                max_linesearch_iters,
                                d_tol
                                )


        self.N = N
        self.nx, self.nu, self.ndx = model.get_dims()

        self.xtraj: np.ndarray = np.zeros((self.N, self.nx))
        self.utraj: np.ndarray = np.zeros((self.N-1, self.nu))
        self.uref: np.ndarray = np.zeros(self.nu, dtype = np.float64)

    def WarmStart(self, init_state, goal_state):
        self.controller.model.set_references(goal_state, self.uref)
        if not self.utraj.any():
            #Normally distribute about a hover @ identity rotation
            self.utraj[:] = np.random.randn(self.N-1, self.nu)*1
        self.xtraj, self.utraj[:] = self.controller.run_ilqr(init_state, self.utraj)


    def DoCalcVectorOutput(self, current_state, goal_state):
        if not np.allclose(self.controller.model.xref, goal_state):
            self.controller.model.set_references(goal_state, self.uref)

        if not self.utraj.any():
            #Normally distribute about a hover @ identity rotation
            self.utraj[:] = 3.08193537 + np.random.randn(self.N-1, self.nu)*1.0
        else:
            #Use previous utraj as initial guess with shifting forward a single timestep, 
            self.utraj[:-1, :] = self.utraj[1:,:]

        self.xtraj, self.utraj[:] = self.controller.run_ilqr(current_state, self.utraj)

        return self.xtraj, self.utraj[0,:]
     
from IPython.display import SVG, display, clear_output, Markdown
from control.models.acrobot import Acrobot2DModel
from control.iLQRController import iLQR

Q = np.diag(np.concatenate(([1.0] * 2, [0.1] * 2)))
Qf = np.diag(np.concatenate(([100.0] * 2, [100.0] * 2)))
R = np.diag([0.05])
dt = 0.05
N = 40

iLQRParams = {
    "Q" : Q,
    "R" : R,
    "Qf" : Qf,
    "N" : N,
    "dt" : dt,
    "max_iter" : 20,
    "max_linesearch_iters": 10,
    "d_tol": 1e-3,
    "integration": "RK4",
}
controller = AcrobotiLQRController2(**iLQRParams)
goal_state  = np.array([np.pi, 0., 0., 0.])
current_state = np.array([0., 0., 0., 0.])
# current_state = np.array([-0.13827868, -1.01150499, -0.41447444,  2.90461387])
# current_state = np.array([ 2.83752567,  0.7953031,   0.0177701,  -0.2012475 ])
# controller.WarmStart(current_state, goal_state)
# controller.utraj = action_log[:N-1]

#Set up simulation
builder = DiagramBuilder()
acrobot = builder.AddSystem(AcrobotPlant())
scene_graph = builder.AddSystem(SceneGraph())
AcrobotGeometry.AddToBuilder(builder, acrobot.get_output_port(0), scene_graph)
meshcat.Delete()
meshcat.Set2dRenderMode(xmin=-4, xmax=4, ymin=-4, ymax=4)
meshcat_visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
diagram = builder.Build()

simulator = Simulator(diagram)
diagram_context = simulator.get_mutable_context()

acrobot_context = acrobot.GetMyContextFromRoot(diagram_context)
control_port = acrobot.get_input_port()
state_port = acrobot.get_output_port()

# Simulate
simulator.set_target_realtime_rate(1.0)
diagram_context.SetTime(0.0)
diagram_context.SetContinuousState(current_state)
simulator.Initialize()
state_error = 0
time = 0
state_log = current_state
action_log = None

meshcat.StartRecording()
while np.linalg.norm(state_error) > 1e-4 or time == 0:
    current_state = state_port.Eval(acrobot_context)
    xtraj, u = controller.DoCalcVectorOutput(current_state, goal_state)
    control_port.FixValue(acrobot_context, u)
    simulator.AdvanceTo(diagram_context.get_time() + dt)

    # Diagnostics
    state_error = current_state - goal_state
    state_error[:2] /= np.pi
    terminal_state_error = xtraj[-1, :] - goal_state
    terminal_state_error[:2] /= np.pi
    # print(xtraj)
    clear_output(wait=True)
    display(Markdown(f"**t = {time:4.4f}s:**\n```\nState: {current_state}\n```\n**Control:**\n```\n{u}\n```\n**Terminal State Err:**\n```\n{terminal_state_error}"))
    time += dt

    state_log = np.vstack((state_log, current_state))
    action_log = np.vstack((action_log, u)) if action_log is not None else u   
meshcat.StopRecording()
meshcat.PublishRecording()

## Drake with DoCalcVectorOutput

In [ ]:
from control.AcrobotControllers import AcrobotiLQRController, AcrobotLQRController
from utils.UtilLeafSystems import NonConstantVectorSource

builder = DiagramBuilder()
acrobot = builder.AddSystem(AcrobotPlant())
goal_state_source = builder.AddSystem(NonConstantVectorSource(4))

Q = np.diag(np.concatenate(([1.0] * 2, [0.1] * 2)))
Qf = np.diag(np.concatenate(([100.0] * 2, [100.0] * 2)))
R = np.diag([0.05])
dt = 0.05
N = 40

iLQRParams = {
    "Q" : Q,
    "R" : R,
    "Qf" : Qf,
    "N" : N,
    "dt" : dt,
    "max_iter" : 20,
    "max_linesearch_iters": 10,
    "d_tol" : 1e-3,
    "integration": "RK4",
}

controller = builder.AddSystem(AcrobotiLQRController(**iLQRParams))
# controller = builder.AddSystem(AcrobotLQRController(**iLQRParams))

builder.Connect(
    acrobot.get_output_port(0), controller.get_input_port(0)
    )
builder.Connect(
    controller.get_output_port(0), acrobot.get_input_port(0)
    )
builder.Connect(
    goal_state_source.get_output_port(), controller.get_input_port(1)
    )
# Setup visualization
scene_graph = builder.AddSystem(SceneGraph())
AcrobotGeometry.AddToBuilder(builder, acrobot.get_output_port(0), scene_graph)
meshcat.Delete()
meshcat.Set2dRenderMode(xmin=-4, xmax=4, ymin=-4, ymax=4)
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

diagram = builder.Build()
init_state = np.array([0.0, 0.0, 0.0, 0.0])
goal_state = np.array([np.pi, 0., 0., 0.])

# controller.WarmStart(init_state, goal_state)
def simulate(init_state, goal_state, diagram, acrobot, controller, duration):
    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)
    integrator = simulator.get_integrator()
    integrator.set_fixed_step_mode(1)
    diagram_context = simulator.get_mutable_context()

    acrobot_context = acrobot.GetMyContextFromRoot(diagram_context)
    controller_context = controller.GetMyContextFromRoot(diagram_context)

    acrobot_state_port = acrobot.get_output_port(0)
    control_output_port = controller.get_output_port(0)


    # Simulate
    simulator.set_target_realtime_rate(1.0)
    goal_state_source.SetState(goal_state)
    diagram_context.SetTime(0.0)
    diagram_context.SetContinuousState(init_state)
    simulator.Initialize()

    while diagram_context.get_time() < duration:
        time = diagram_context.get_time()
        simulator.AdvanceTo(diagram_context.get_time() + dt)
        state = acrobot_state_port.Eval(acrobot_context)
        control = control_output_port.Eval(controller_context)
        clear_output(wait=True)
        display(Markdown(f"**t = {time:4.4f}s:**\n```\nState: {state}\n```\n**Control:**\n```\n{control}\n```\n"))
    simulator.AdvanceTo(duration)

simulate(init_state, goal_state, diagram, acrobot, controller, 10.0)